In [15]:
import tensorflow as tf
import json
import os
import math
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import glob
import numpy as np


In [4]:
# DATA MAPPING

# Define the folders
folders = {"Blanks": "small_animals/Blanks", "American_Bullfrog": "small_animals/American_Bullfrog", 
           "American_Toad": "small_animals/American_Toad", "Green_Frog": "small_animals/Green_Frog", 
           "Northern_Leopard_Frog": "small_animals/Northern_Leopard_Frog", "Common_Yellowthroat": "small_animals/Common_Yellowthroat",
           "Eastern_Bluebird": "small_animals/Eastern_Bluebird", "Gray_Catbird": "small_animals/Gray_Catbird",
           "Indigo_Bunting": "small_animals/Indigo_Bunting", "Northern_House_Wren": "small_animals/Northern_House_Wren",
           "Song_Sparrow": "small_animals/Song_Sparrow", "Sora": "small_animals/Sora", "Invertebrate": "small_animals/Invertebrate",
           "Common_Five-linked_skink": "small_animals/Common_Five-linked_skink", "American_Mink": "small_animals/American_Mink",
           "Brown_Rat": "small_animals/Brown_Rat", "Eastern_Chipmunk": "small_animals/Eastern_Chipmunk",
           "Eastern_Cottontail": "small_animals/Eastern_Cottontail", "Long_tailed_Weasel": "small_animals/Long_tailed_Weasel",
           "Masked_Shrew": "small_animals/Masked_Shrew", "Meadow_Jumping_Mouse": "small_animals/Meadow_Jumping_Mouse",
           "Meadow_Vole": "small_animals/Meadow_Vole", "N._Short-tailed_Shrew": "small_animals/N._Short-tailed_Shrew",
           "Raccoon": "small_animals/Raccoon", "Star-nosed_mole": "small_animals/Star-nosed_mole",
           "Striped_Skunk": "small_animals/Striped_Skunk", "Virginia_Opossum": "small_animals/Virginia_Opossum",
           "White-footed_Mouse": "small_animals/White-footed_Mouse", "Woodchuck": "small_animals/Woodchuck",
           "Woodland_Jumping_Mouse": "small_animals/Woodland_Jumping_Mouse", "Butler's_Gartersnake": "small_animals/Butler's_Gartersnake",
           "Dekay's_Brownsnake": "small_animals/Dekay's_Brownsnake", "Eastern_Gartersnake": "small_animals/Eastern_Gartersnake",
           "Eastern_Hog-nosed_snake": "small_animals/Eastern_Hog-nosed_snake", "Eastern_Massasauga": "small_animals/Eastern_Massasauga",
           "Eastern_Milksnake": "small_animals/Eastern_Milksnake", "Eastern_Racer_Snake": "small_animals/Eastern_Racer_Snake",
           "Eastern_Ribbonsnake": "small_animals/Eastern_Ribbonsnake", "Gray_Ratsnake": "small_animals/Gray_Ratsnake",
           "Kirtland's_Snake": "small_animals/Kirtland's_Snake", "Northern_Watersnake": "small_animals/Northern_Watersnake",
           "Plains_Gartersnake": "small_animals/Plains_Gartersnake", "Red-bellied_Snake": "small_animals/Red-bellied_Snake",
           "Smooth_Greensnake": "small_animals/Smooth_Greensnake", "Painted_Turtle": "small_animals/Painted_Turtle",
           "Snapping_Turtle": "small_animals/Snapping_Turtle"
        }

# Define categories
blanks = {"Blanks"}
invertebrates = {"Invertebrate"}
lizards = {"Common_Five-linked_skink"}
turtles = {"Painted_Turtle", "Snapping_Turtle" }
amphibians = {"American_Bullfrog", "American_Toad", "Green_Frog", "Northern_Leopard_Frog" }
birds = {"Common_Yellowthroat", "Eastern_Bluebird", "Gray_Catbird", "Indigo_Bunting", 
         "Northern_House_Wren", "Song_Sparrow", "Sora"}
mammals = {"American_Mink", "Brown_Rat", "Eastern_Chipmunk", "Eastern_Cottontail", "Long_tailed_Weasel",
           "Masked_Shrew", "Meadow_Jumping_Mouse", "Meadow_Vole", "N._Short-tailed_Shrew", "Raccoon", 
           "Star-nosed_mole", "Striped_Skunk", "Virginia_Opossum", "White-footed_Mouse", "Woodchuck",
           "Woodland_Jumping_Mouse" }
snakes = {"Butler's_Gartersnake", "Dekay's_Brownsnake", "Eastern_Gartersnake", "Eastern_Hog-nosed_snake",
          "Eastern_Massasauga", "Eastern_Milksnake", "Eastern_Racer_Snake", "Eastern_Ribbonsnake", 
          "Gray_Ratsnake", "Kirtland's_Snake", "Northern_Watersnake", "Plains_Gartersnake", 
          "Red-bellied_Snake", "Smooth_Greensnake"}

# Dictionary to hold the file paths with their labels
file_paths_with_labels = []

# Iterate through each folder
for label, folder_path in folders.items():
    # Get all file paths in the folder
    file_paths = glob.glob(os.path.join(folder_path, "*"))
    
    # Append the file paths with their labels
    file_paths_with_labels.extend([(file_path, label) for file_path in file_paths])


In [7]:
# Print sample file paths and labels to ensure correctness

print(file_paths_with_labels[1500:1502])
print(file_paths_with_labels[15000:15002])
print(file_paths_with_labels[15000:15002])
print(file_paths_with_labels[50000:50002])
print(file_paths_with_labels[100000:100002])

[('small_animals/Blanks/CBNP1N_2020-09-14_20-27-50.JPG', 'Blanks'), ('small_animals/Blanks/CBNP1S_2020-10-22_10-13-15.JPG', 'Blanks')]
[('small_animals/Northern_House_Wren/FCM3__2019-08-29__11-28-44(7).JPG', 'Northern_House_Wren'), ('small_animals/Northern_House_Wren/FCM1__2019-08-18__12-16-28(2).JPG', 'Northern_House_Wren')]
[('small_animals/Northern_House_Wren/FCM3__2019-08-29__11-28-44(7).JPG', 'Northern_House_Wren'), ('small_animals/Northern_House_Wren/FCM1__2019-08-18__12-16-28(2).JPG', 'Northern_House_Wren')]
[('small_animals/Masked_Shrew/NOR3__2019-06-01__19-29-18(4).JPG', 'Masked_Shrew'), ('small_animals/Masked_Shrew/FCM1__2019-06-14__06-57-41(1).JPG', 'Masked_Shrew')]
[('small_animals/Eastern_Gartersnake/NOR3__2019-08-31__16-07-08(5).JPG', 'Eastern_Gartersnake'), ('small_animals/Eastern_Gartersnake/KILC4S__2022-10-03__15-23-57(3)__Thamnophis_sirtalis.JPG', 'Eastern_Gartersnake')]


In [ ]:
# RESNET MODEL HYPERPARAMETERS AND TENSORFLOW DATASET

# Define hyperparameters
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 32
EPOCHS = 10
FINE_TUNE_EPOCHS = 5  # Fine-tuning epochs

# Split data into features (file paths) and labels
file_paths, labels = zip(*file_paths_with_labels)

# Split into training and testing data
train_file_paths, test_file_paths, train_labels, test_labels = train_test_split(file_paths, labels, test_size=0.2, random_state=123)

print("Training set size:", len(train_file_paths))
print("Test set size:", len(test_file_paths))

# Calculate steps per epoch, rounding up
steps_per_epoch = math.floor(len(train_file_paths) / BATCH_SIZE)

# Convert labels to integers using the label map
label_map = {label: idx for idx, label in enumerate(set(labels))}
train_labels = [label_map[label] for label in train_labels]
test_labels = [label_map[label] for label in test_labels]

# Create TensorFlow dataset from file paths and labels
def create_tf_dataset(file_paths, labels, batch_size):
    def parse_image(file_path, label):
        try:
            # Read the image from file
            img = tf.io.read_file(file_path)
            # Decode the image
            img = tf.image.decode_jpeg(img, channels=3)
            # Resize the image to target size
            img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
            # Normalize pixel values to [0, 1]
            img = img / 255.0
            return img, label
        except tf.errors.InvalidArgumentError:
            # Log the error and return None to indicate skipping this image
            print(f"Error reading image: {file_path}. Skipping this image.")
            return None  # Indicate this image should be skipped

    # Create a TensorFlow dataset
    ds = tf.data.Dataset.from_tensor_slices((file_paths, labels))

    # Remove invalid images (those that returned None) using the filter meth od
    ds = ds.map(parse_image, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.filter(lambda img, label: img is not None)  # Filter out None images

    # Shuffle, batch, and prefetch the dataset
    ds = ds.shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return ds

# Create training and testing datasets
train_dataset = create_tf_dataset(train_file_paths, train_labels, BATCH_SIZE)
test_dataset = create_tf_dataset(test_file_paths, test_labels, BATCH_SIZE)

# Repeats indefinitely to avoid errors
train_dataset = train_dataset.repeat()  


In [9]:
# RESNET MODEL BEFORE FINE TUNING

# Load pre-trained ResNet50 model (without top layer)
base_model = tf.keras.applications.ResNet50(
    weights='imagenet',  # Load pre-trained weights from ImageNet
    include_top=False,   # Do not include the final fully connected layer, because I will add one later
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# Freeze the layers of ResNet50 for first round, before fine tuning
base_model.trainable = False

# Build the model
model = models.Sequential([
    base_model,  # Add the pre-trained ResNet50 as the base model
    layers.GlobalAveragePooling2D(),  # Add a global average pooling layer
    layers.Dense(128, activation='relu'),  # Add a fully connected layer
    layers.Dense(len(label_map), activation='softmax')  # Output layer for classification
])

# Compile the model for the first bit with the frozen layers
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with the frozen base layers
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch
)



Training set size: 92920
Test set size: 23230
Epoch 1/10
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 3943s 1s/step - accuracy: 0.3441 - loss: 2.2252 - val_accuracy: 0.4764 - val_loss: 1.7732
Epoch 2/10


2024-12-08 18:21:17.512619: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


2903/2903 ━━━━━━━━━━━━━━━━━━━━ 5470s 2s/step - accuracy: 0.4714 - loss: 1.7764 - val_accuracy: 0.4888 - val_loss: 1.6891
Epoch 3/10


2024-12-08 19:52:27.953352: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2903/2903 ━━━━━━━━━━━━━━━━━━━━ 641s 221ms/step - accuracy: 0.4907 - loss: 1.6752 - val_accuracy: 0.5138 - val_loss: 1.5878
Epoch 4/10
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 654s 225ms/step - accuracy: 0.5057 - loss: 1.6060 - val_accuracy: 0.5241 - val_loss: 1.5289
Epoch 5/10


2024-12-08 20:14:02.815507: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2903/2903 ━━━━━━━━━━━━━━━━━━━━ 680s 234ms/step - accuracy: 0.5211 - loss: 1.5540 - val_accuracy: 0.5304 - val_loss: 1.5010
Epoch 6/10
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 1036s 357ms/step - accuracy: 0.5318 - loss: 1.5164 - val_accuracy: 0.5483 - val_loss: 1.4696
Epoch 7/10
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 680s 234ms/step - accuracy: 0.5425 - loss: 1.4820 - val_accuracy: 0.5621 - val_loss: 1.4314
Epoch 8/10
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 686s 236ms/step - accuracy: 0.5522 - loss: 1.4524 - val_accuracy: 0.5614 - val_loss: 1.4154
Epoch 9/10
   1/2903 ━━━━━━━━━━━━━━━━━━━━ 7:56 164ms/step - accuracy: 0.6250 - loss: 1.2455

2024-12-08 21:05:25.351335: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2903/2903 ━━━━━━━━━━━━━━━━━━━━ 3118s 1s/step - accuracy: 0.5607 - loss: 1.4291 - val_accuracy: 0.5662 - val_loss: 1.3907
Epoch 10/10
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 1781s 614ms/step - accuracy: 0.5671 - loss: 1.4065 - val_accuracy: 0.5778 - val_loss: 1.3859
726/726 ━━━━━━━━━━━━━━━━━━━━ 129s 176ms/step


NameError: name 'np' is not defined

In [12]:
# Evaluate the model before fine tuning 

# Get predictions for the test dataset
predictions = model.predict(test_dataset)
predictions = np.argmax(predictions, axis=1)

# Get the true labels from the test dataset
true_labels = np.array(test_labels)

# Print classification report
print("Classification Report before fine tuning:")
print(classification_report(true_labels, predictions, target_names=list(label_map.keys())))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy before fine tuning: {test_accuracy:.2f}")



726/726 ━━━━━━━━━━━━━━━━━━━━ 129s 176ms/step
Classification Report before fine tuning:


NameError: name 'classification_report' is not defined

In [17]:
# FIRST ROUND OF FINE TUNING

# Unfreeze the top layers of the ResNet50 model for fine-tuning
base_model.trainable = True

# Unfreeze only the last 25 layers
for layer in base_model.layers[:-25]:
    layer.trainable = False

# Re-compile the model with a smaller learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Continue training the model with fine-tuning
history_finetune = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=FINE_TUNE_EPOCHS,  # Fine-tune for additional epochs
    steps_per_epoch=steps_per_epoch
)

# Evaluate the model after fine tuning 

# Get predictions for the test dataset
predictions = model.predict(test_dataset)
predictions = np.argmax(predictions, axis=1)

# Get the true labels from the test dataset
true_labels = np.array(test_labels)

# Print classification report
print("Classification Report after fine tuning:")
print(classification_report(true_labels, predictions, target_names=list(label_map.keys())))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy after fine tuning: {test_accuracy:.2f}")

# Save the model
tf.keras.models.save_model(model, 'ResNet_model.keras')


Classification Report before fine tuning:
                          precision    recall  f1-score   support

    Butler's_Gartersnake       0.00      0.00      0.00        26
              Green_Frog       0.00      0.00      0.00        15
            Masked_Shrew       0.02      0.01      0.01       837
          Indigo_Bunting       0.00      0.00      0.00         7
      Plains_Gartersnake       0.00      0.00      0.00       237
       Smooth_Greensnake       0.00      0.00      0.00        33
       Eastern_Milksnake       0.00      0.00      0.00        10
Common_Five-linked_skink       0.04      0.03      0.03      1003
               Woodchuck       0.00      0.00      0.00         9
     Eastern_Ribbonsnake       0.00      0.00      0.00        24
 Eastern_Hog-nosed_snake       0.00      0.00      0.00        12
      Dekay's_Brownsnake       0.00      0.00      0.00        92
     Northern_House_Wren       0.05      0.11      0.07      1207
        Virginia_Opossum       1.

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


726/726 ━━━━━━━━━━━━━━━━━━━━ 128s 175ms/step - accuracy: 0.5806 - loss: 1.3757
Test Accuracy before fine tuning: 0.58
Epoch 1/5


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


2903/2903 ━━━━━━━━━━━━━━━━━━━━ 2138s 735ms/step - accuracy: 0.5423 - loss: 3.3560 - val_accuracy: 0.6606 - val_loss: 1.1561
Epoch 2/5
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 926s 319ms/step - accuracy: 0.6719 - loss: 1.1287 - val_accuracy: 0.7051 - val_loss: 1.0133
Epoch 3/5
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 929s 320ms/step - accuracy: 0.7032 - loss: 1.0049 - val_accuracy: 0.7158 - val_loss: 0.9713
Epoch 4/5


...

2903/2903 ━━━━━━━━━━━━━━━━━━━━ 4242s 1s/step - accuracy: 0.7278 - loss: 0.9055 - val_accuracy: 0.7367 - val_loss: 0.8935
Epoch 5/5
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 10964s 4s/step - accuracy: 0.7513 - loss: 0.8248 - val_accuracy: 0.7238 - val_loss: 0.9375
726/726 ━━━━━━━━━━━━━━━━━━━━ 159s 216ms/step
Classification Report after fine tuning:
                          precision    recall  f1-score   support

    Butler's_Gartersnake       0.00      0.00      0.00        26
              Green_Frog       0.00      0.00      0.00        15
            Masked_Shrew       0.06      0.02      0.03       837
          Indigo_Bunting       0.00      0.00      0.00         7
      Plains_Gartersnake       0.01      0.00      0.01       237
       Smooth_Greensnake       0.00      0.00      0.00        33
       Eastern_Milksnake       0.00      0.00      0.00        10
Common_Five-linked_skink       0.04      0.05      0.04      1003
               Woodchuck       0.00      0.00      0.00         9


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


726/726 ━━━━━━━━━━━━━━━━━━━━ 1082s 1s/step - accuracy: 0.7233 - loss: 0.9408
Test Accuracy after fine tuning: 0.72


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


NameError: name 'tensorflow' is not defined

In [20]:
# Save the model
tf.keras.models.save_model(model, 'ResNet_model.keras')


In [22]:
# SECOND ROUND OF FINE TUNING

# Unfreeze the top layers of the ResNet50 model for fine-tuning
base_model.trainable = True

# Unfreeze only the last 50 layers
for layer in base_model.layers[:-50]:
    layer.trainable = False

# Re-compile the model with a smaller learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Continue training the model with fine-tuning
history_finetune = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=FINE_TUNE_EPOCHS,  # Fine-tune for additional epochs
    steps_per_epoch=steps_per_epoch
)

# Evaluate the model after fine tuning 

# Get predictions for the test dataset
predictions = model.predict(test_dataset)
predictions = np.argmax(predictions, axis=1)

# Get the true labels from the test dataset
true_labels = np.array(test_labels)

# Print classification report
print("Classification Report after fine tuning:")
print(classification_report(true_labels, predictions, target_names=list(label_map.keys())))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy after fine tuning: {test_accuracy:.2f}")

# Save the model
tf.keras.models.save_model(model, 'ResNet_model.keras')

Epoch 1/5
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 1074s 369ms/step - accuracy: 0.7698 - loss: 0.7560 - val_accuracy: 0.7569 - val_loss: 0.8526
Epoch 2/5
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 1903s 655ms/step - accuracy: 0.7883 - loss: 0.6890 - val_accuracy: 0.7497 - val_loss: 0.8022
Epoch 3/5
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 941s 324ms/step - accuracy: 0.8019 - loss: 0.6361 - val_accuracy: 0.8026 - val_loss: 0.6741
Epoch 4/5
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 947s 326ms/step - accuracy: 0.8135 - loss: 0.5965 - val_accuracy: 0.7994 - val_loss: 0.6688
Epoch 5/5
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 955s 329ms/step - accuracy: 0.8267 - loss: 0.5474 - val_accuracy: 0.8046 - val_loss: 0.6682
726/726 ━━━━━━━━━━━━━━━━━━━━ 134s 182ms/step
Classification Report after fine tuning:
                          precision    recall  f1-score   support

    Butler's_Gartersnake       0.00      0.00      0.00        26
              Green_Frog       0.00      0.00      0.00        15
            Masked_Shrew       0.03      0.02   

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


726/726 ━━━━━━━━━━━━━━━━━━━━ 132s 180ms/step - accuracy: 0.8101 - loss: 0.6560
Test Accuracy after fine tuning: 0.80


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


In [24]:
# THIRD ROUND OF FINE TUNING

# Unfreeze the top layers of the ResNet50 model for fine-tuning
base_model.trainable = True

# Unfreeze only the last 100 layers
for layer in base_model.layers[:-100]:
    layer.trainable = False

# Re-compile the model with a smaller learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Continue training the model with fine-tuning
history_finetune = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=FINE_TUNE_EPOCHS,  # Fine-tune for additional epochs
    steps_per_epoch=steps_per_epoch
)

# Evaluate the model after fine tuning 

# Get predictions for the test dataset
predictions = model.predict(test_dataset)
predictions = np.argmax(predictions, axis=1)

# Get the true labels from the test dataset
true_labels = np.array(test_labels)

# Print classification report
print("Classification Report after fine tuning:")
print(classification_report(true_labels, predictions, target_names=list(label_map.keys())))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy after fine tuning: {test_accuracy:.2f}")

# Save the model
tf.keras.models.save_model(model, 'ResNet_model.keras')

Epoch 1/5
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 891s 306ms/step - accuracy: 0.8365 - loss: 0.5155 - val_accuracy: 0.8144 - val_loss: 0.6229
Epoch 2/5
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 2156s 743ms/step - accuracy: 0.8460 - loss: 0.4831 - val_accuracy: 0.8269 - val_loss: 0.5927
Epoch 3/5
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 915s 315ms/step - accuracy: 0.8569 - loss: 0.4502 - val_accuracy: 0.8207 - val_loss: 0.5985
Epoch 4/5
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 926s 319ms/step - accuracy: 0.8644 - loss: 0.4234 - val_accuracy: 0.8347 - val_loss: 0.5606
Epoch 5/5
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 1602s 552ms/step - accuracy: 0.8728 - loss: 0.3976 - val_accuracy: 0.8392 - val_loss: 0.5373


2024-12-09 13:30:12.236484: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


726/726 ━━━━━━━━━━━━━━━━━━━━ 129s 175ms/step
Classification Report after fine tuning:
                          precision    recall  f1-score   support

    Butler's_Gartersnake       0.00      0.00      0.00        26
              Green_Frog       0.00      0.00      0.00        15
            Masked_Shrew       0.05      0.05      0.05       837
          Indigo_Bunting       0.00      0.00      0.00         7
      Plains_Gartersnake       0.01      0.00      0.01       237
       Smooth_Greensnake       0.00      0.00      0.00        33
       Eastern_Milksnake       0.00      0.00      0.00        10
Common_Five-linked_skink       0.04      0.04      0.04      1003
               Woodchuck       0.00      0.00      0.00         9
     Eastern_Ribbonsnake       0.00      0.00      0.00        24
 Eastern_Hog-nosed_snake       0.00      0.00      0.00        12
      Dekay's_Brownsnake       0.00      0.00      0.00        92
     Northern_House_Wren       0.06      0.06      0.06

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


726/726 ━━━━━━━━━━━━━━━━━━━━ 130s 178ms/step - accuracy: 0.8414 - loss: 0.5341
Test Accuracy after fine tuning: 0.84


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


In [26]:
# FOURTH ROUND OF FINE TUNING

# Unfreeze the top layers of the ResNet50 model for fine-tuning
base_model.trainable = True

# Unfreeze only the last 200 layers
for layer in base_model.layers[:-200]:
    layer.trainable = False

# Re-compile the model with a smaller learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Continue training the model with fine-tuning
history_finetune = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=FINE_TUNE_EPOCHS,  # Fine-tune for additional epochs
    steps_per_epoch=steps_per_epoch
)

# Evaluate the model after fine tuning 

# Get predictions for the test dataset
predictions = model.predict(test_dataset)
predictions = np.argmax(predictions, axis=1)

# Get the true labels from the test dataset
true_labels = np.array(test_labels)

# Print classification report
print("Classification Report after fine tuning:")
print(classification_report(true_labels, predictions, target_names=list(label_map.keys())))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy after fine tuning: {test_accuracy:.2f}")

# Save the model
tf.keras.models.save_model(model, 'ResNet_model.keras')

Epoch 1/5
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 911s 312ms/step - accuracy: 0.8776 - loss: 0.3762 - val_accuracy: 0.8449 - val_loss: 0.5109
Epoch 2/5
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 929s 320ms/step - accuracy: 0.8846 - loss: 0.3544 - val_accuracy: 0.8423 - val_loss: 0.5148
Epoch 3/5
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 1494s 515ms/step - accuracy: 0.8925 - loss: 0.3326 - val_accuracy: 0.8415 - val_loss: 0.5646
Epoch 4/5
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 909s 313ms/step - accuracy: 0.8962 - loss: 0.3158 - val_accuracy: 0.8256 - val_loss: 0.5937
Epoch 5/5
2903/2903 ━━━━━━━━━━━━━━━━━━━━ 1543s 532ms/step - accuracy: 0.9013 - loss: 0.2992 - val_accuracy: 0.8517 - val_loss: 0.5197
726/726 ━━━━━━━━━━━━━━━━━━━━ 131s 178ms/step
Classification Report after fine tuning:
                          precision    recall  f1-score   support

    Butler's_Gartersnake       0.00      0.00      0.00        26
              Green_Frog       0.00      0.00      0.00        15
            Masked_Shrew       0.04      0.04   

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


726/726 ━━━━━━━━━━━━━━━━━━━━ 201s 275ms/step - accuracy: 0.8553 - loss: 0.5146
Test Accuracy after fine tuning: 0.85


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()
